# 워드투벡터(Word2Vec)


*   (원-핫 벡터와 달리) 단어 간 유사도를 반영할 수 있도록 단어의 의미를 벡터화 하는 방법이다.



## CBOW(Continuous Bag of Words)
- 주변에 있는 단어를 자기고 중간에 있는 단어들을 예측하는 방법이다.

      The fat cat sat on the mat
      {'The', 'fat', 'cat', 'on', 'the', 'mat'} → {'sat'}을 예측해야 한다.

      # 예측해야 하는 단어('sat')를 중심 단어(center word)라 한다.
      # 예측에 사용되는 단어들을 주변 단어(context word)라 한다.

      # 중심 단어를 예측하기 위해서 앞,뒤로 몇개의 단어를 볼지 결정하고, 이 범위를 윈도우(window)라 한다.
      # 윈도우의 크기 2 : 중심 단어('sat)를 예측하기 위해서 앞의 두 단어('fat', 'cat'), 뒤의 두 단어('on', 'the')를 참고한다.

*  **슬라이딩 윈도우(sliding window)**: 윈도우를 계속 움직여가며, 주변 단어와 중심 단어를 바꿔가며 학습을 위한 데이터 셋을 만드는 방법이다.

![대체 텍스트](https://wikidocs.net/images/page/22660/%EB%8B%A8%EC%96%B4.PNG)

→ 위의 그림은 윈도우 크기를 2로 정하여 어떻게 슬라이딩 윈도우가 이루어지는 지를 보여주고 있다.

→ 오른쪽 표와 같이, 워드투벡터에서 입력은 모두 원-핫 벡터가 되어야 한다.

# 워드투벡터 실습
##[ 데이터 로드 및 전처리 ]

In [ ]:
import re
from lxml import etree
import urllib.request
import zipfile
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")
# 데이터 다운로드

with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
  target_text = etree.parse(z.open('ted_en-20160408.xml', 'r'))
  parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

In [ ]:
parse_text[:300] # 로드한 데이터에서 300개의 글자(character)만 출력

"Here are two reasons companies fail: they only do more of the same, or they only do what's new.\nTo me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.\nConsider Facit"

In [ ]:
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sent_text=sent_tokenize(content_text)

In [ ]:
normalized_text=[]
for string in sent_text:
  tokens=re.sub(r"[^a-z0-9]+", " ", string.lower())  # 각 문장에 대해서 구두점 제거하고, 대문자를 소문자로 변환
  normalized_text.append(tokens)

result=[word_tokenize(sentence) for sentence in normalized_text]  # 각 문장에 대해서 NLTK를 이용하여 단어 토큰화 수행

In [ ]:
print('총 샘플의 개수: {}'.format(len(result)))

총 샘플의 개수: 273424


In [ ]:
for line in result[:1]:
  print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']




*   상위 1개 문장 출력을 통해서, 문장에 대해 토큰화가 수행되었음을 확인할 수 있다.



## [ Word2Vec 훈련시키기 ]

In [ ]:
from gensim.models import Word2Vec
model=Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

# size : 임베딩 된 벡터의 차원
# window : 컨텍스트 윈도우 크기
# min_count : 단어의 최소 빈도수 제한(빈도가 적은 단어들은 학습하지 않는다.)
# workers : 학습을 위한 프로세스 수
# sg=0 : CBOW

In [ ]:
# model.wv.most_similar: 입력한 단어에 대해서 가장 유사한 단어를 출력하는 모듈
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8441615104675293), ('guy', 0.8088982105255127), ('lady', 0.7658368945121765), ('boy', 0.7520817518234253), ('girl', 0.742436408996582), ('soldier', 0.7382450103759766), ('gentleman', 0.7178536057472229), ('kid', 0.7093591690063477), ('poet', 0.6823942065238953), ('king', 0.6648910045623779)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):




*  'man'과 유사한 단어로 {'woman', 'guy', 'lady', 'boy', 'girl', 'soldier', ... } 가 출력되는 것을 알 수 있다.



## [ Word2Vec 모델 저장하고 로드하기 ]

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v')
loaded_model=KeyedVectors.load_word2vec_format("eng_w2v")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
